In [1]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [2]:
get_ipython().run_cell_magic('capture', '', "#- INSTALL ADDITIONAL LIBRARIES IF REQUIRED\n#------------------------------------------\n# This is normally not required. The hub environment comes preinstaled with \n# many packages that you can already use without setup. In case there is some\n# other library you would like to use that isn't on the list you run this command\n# once to install them.  If it is already installed this command has no effect.\n!pip install google-cloud-bigquery\n!pip install google-cloud-bigquery[pandas]\n")

In [3]:
#- IMPORT THE LIBRARIES YOU WILL USE
#------------------------------------------
# You only need to import packages one time per notebook session. To keep your
# notebook clean and organized you can handle all imports at the top of your file.
# The following are included for example purposed, feel free to modify or delete 
# anything in this section.
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import matplotlib.pyplot as plt  
import statsmodels.api as sm
import itertools
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [4]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [5]:
#query 3: overview of employment_data(week 41)
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
ORDER BY week_number ASC;
"""
query_job = bigquery_client.query(query)
overview = query_job.to_dataframe()
overview.head()

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,65f24aac652de33abaf3f227b41e2641,20220101,1,18093,950500,"Census Tract 9505, Lawrence County, Indiana",10,0,<NA>,<NA>,...,<NA>,<NA>,0,0,0,<NA>,0,0,0,<NA>
1,14ceb8012cb1b912044297c8933cfcfc,20220101,1,18057,110506,"Census Tract 1105.06, Hamilton County, Indiana",39,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,<NA>,<NA>,0,<NA>,28
2,f013068de98db1470bd986137a0c6d23,20220101,1,18003,900,"Census Tract 9, Allen County, Indiana",10,0,0,<NA>,...,<NA>,<NA>,0,0,0,<NA>,0,0,<NA>,<NA>
3,f70960d492c2fd067680a14baa0f66e9,20220101,1,18167,10600,"Census Tract 106, Vigo County, Indiana",35,0,<NA>,21,...,<NA>,<NA>,0,0,0,0,<NA>,0,<NA>,32
4,778946d9696cd6c95f57e8b1e6336389,20220101,1,18167,11200,"Census Tract 112, Vigo County, Indiana",37,0,<NA>,24,...,26,11,0,0,0,<NA>,<NA>,0,<NA>,<NA>


In [6]:
#query 2: overview of important info from unemployment table
query = """
SELECT uu_id, week_number, total_claims
FROM `ironhacks-data.ironhacks_competition.unemployment_data` 
ORDER BY week_number ASC
"""
query_jobb = bigquery_client.query(query)
employ = query_jobb.to_dataframe()
print(employ.head())

                              uu_id  week_number  total_claims
0  65f24aac652de33abaf3f227b41e2641            1            10
1  14ceb8012cb1b912044297c8933cfcfc            1            39
2  f013068de98db1470bd986137a0c6d23            1            10
3  f70960d492c2fd067680a14baa0f66e9            1            35
4  778946d9696cd6c95f57e8b1e6336389            1            37


In [7]:
#query 3: overview of prediction list (week 41)
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""
query_job = bigquery_client.query(query)
predn = query_job.to_dataframe()
predn.head()
print(predn.head())

                              uu_id  week_number
0  5bf51fc2e162d6faf9e3cf79e4198378           44
1  420b44cc7e3f55d738df565421e59941           44
2  e39c66ecceec76ee8f9f811fa4a2d246           44
3  a90462cd11ae4e43144239bf7c4828a4           44
4  8b20a6749088c7ff1237983076ebfeaa           44
